## Fixing the nth power function
### Previous version
This version was very slow because it was using additional steps, even compared to the most basic algorithm of just using multiplication n times. Additionally, even multiplying a LazySeries object by 1 was computing time wise almost equivalent to two LazySeries objects being multiplied which is why we removed the first step by implementing a while loop to find the first digit equal to 1 in the binary representation of n. 
Here is the old code:
```
-- Raising LazySeries by nth power
LazySeries ^ ZZ := LazySeries => (S,n) -> ( -- TAKING TOO LONG TO COMPUTE
    R := S#seriesRing;
    if n == 0 then return oneSeries(R);
    if n == 1 then  return S;

    finalResult := 1; -- prints this one, UPDATE: CHANGED IT TO 1 
    bin := toBinary(n);
    tempCalculation := S;
    binDigit := 0;

    for i from 0 to #bin-1 when i>=0 do(
        print i;
        binDigit = bin#(#bin-1-i);
        if (binDigit == 1) then (
            if (finalResult == 1) then finalResult = tempCalculation
            else finalResult = finalResult * tempCalculation;
            );

        if (i < #bin-1) then
            tempCalculation = tempCalculation * tempCalculation;
    );
    if debugLevel > 0 then print "FINAL RESULT:";
    finalResult
);
```
### New version
Here is the newer faster version:
```
-- Raising LazySeries by nth power
LazySeries ^ ZZ := LazySeries => (S,n) -> (
    R := S#seriesRing;
    if n == 0 then return oneSeries(R);
    if n == 1 then  return S;

    bin := toBinary(n);
    finalResult := 1;
    tempCalculation:= S;
    j := 1;
    while (true) do (
        -- print j;        
        if(bin#(#bin-j)== 1) then (
            finalResult = tempCalculation;
            break;
        );        
        tempCalculation = tempCalculation * tempCalculation;
        -- print "1 CALCULATION z";
        j = j+1;
    );
    -- << "final j: "<< j <<endl;

    for i from 0 to #bin-j-1 when i >= 0 do(
        -- << "#bin-j-1-i: " << #bin-j-1-i<< endl;
        tempCalculation = tempCalculation * tempCalculation; 
        -- print "1 CALCULATION y";
        if bin#(#bin-j-1-i) == 1 then (
            finalResult = finalResult * tempCalculation;
            -- print "1 CALCULATION x";
        );       
    );
    finalResult
);
```

## `pretty` for LazySeries


In [15]:
loadPackage "NewPowerSeries"
R = QQ[x,y,z]
f = {i,j,k}-> 7
S = lazySeries(R, f)
pretty S


                              2            2                   2       3     2 
o15 = 7 + (7x + 7y + 7z) + (7x  + 7x*y + 7y  + 7x*z + 7y*z + 7z ) + (7x  + 7x y
      --------------------------------------------------------------------------
            2     3     2               2        2       2     3
      + 7x*y  + 7y  + 7x z + 7x*y*z + 7y z + 7x*z  + 7y*z  + 7z )


## Some polynomial methods
- `degree`
- `select`

In [21]:
P = (17*x^6 + 98*x^7*y^8 +87*y^5*x^4)
degree (17*x^6 + 98*x^7*y^8 +87*y^5*x^4)


o21 = {15}

o21 : List


In [22]:
select(P, i -> degree i > {6,5})


         7 8      4 5      6
o22 = 98x y  + 87x y  + 17x

o22 : R


In [23]:
select(P, i -> degree i > {6})


         7 8      4 5      6
o23 = 98x y  + 87x y  + 17x

o23 : R


## Better style
changed
``` 
vars(1..(numgens R))
```
to 
```
gens R
```

## lazySeries(RingElement)
### Old code
```
-- Converting ring elements and polynomials into LazySeries
lazySeries(RingElement) := LazySeries => opts -> P -> (-- Isn't outputting correct degree for the polynomial
    listPolynomial := listForm P; -- USES EXTRA MEMORY AND COMPUTING ITME
    R := ring P;
    variables := gens R;

    h := variables ->(
    for i from 0 to #listPolynomial-1 do(
        
        if variables == (toSequence(listPolynomial#i)#0) then (
            
            return (listPolynomial#i)#1;
            )
        );
        0
    );
    lazySeries(R, h, displayedDegree => sum (degree P), computedDegree => sum (degree P)) -- temporary fix, but will have to make list degrees work in general
);
```
### new code (Faster)
lazySeries(RingElement) := LazySeries => opts -> P -> ( 
    R := ring P;
    variables := gens R;
    newFunction := variables -> coefficient(variables, P);

    lazySeries(R, newFunction, displayedDegree =>sum (degree P), computedDegree => sum (degree P)) 
);

## LazySeries Multiplication
### Old code
Used the newFunction:
```

    newFunction := coefficientVector -> (
        s := 0;
        L := toList ringZeroes .. toList coefficientVector;
        for i from 0 to #L-1 do
            s = s + ((f toSequence(L#i)) * g toSequence(toList coefficientVector -  (L#i)));    
        s
    );
    
```
### NewCode (Faster)
```
LazySeries * LazySeries := LazySeries => (A,B) -> (
    if not (A#seriesRing === B#seriesRing) then error "Rings of series do not match"; -- checks if using same ring

    f := A#coefficientFunction;
    g := B#coefficientFunction;
    R := A#seriesRing;
    ringZeroes := numgens R:0;

    newDegree := max(A.cache.DisplayedDegree, B.cache.DisplayedDegree);
    --newDegree := min(A.cache.DisplayedDegree + B.cache.DisplayedDegree, 10);

    newFunction := coefficientVector -> (
        tempDegree := coefficientVector; -- bandaid!!!!!
        if(class coefficientVector === List) then tempDegree = sum coefficientVector;
        
        a := changeComputedDegree(A, tempDegree);
        b := changeComputedDegree(B, tempDegree);

        P1 := a.cache.computedPolynomial;
        P2 := b.cache.computedPolynomial;

        P := truncate(newDegree, P1*P2);

        coefficient(coefficientVector, P)
    );

    changeComputedDegree(A, newDegree);
    changeComputedDegree(B, newDegree);
    newPoly := truncate(newDegree, (truncate(newDegree, A.cache.computedPolynomial))*(truncate(newDegree, B.cache.computedPolynomial)));

    finalSeries := lazySeries(
        R,
        newFunction,
        newPoly,
        newPoly,
        DisplayedDegree =>  newDegree,
        ComputedDegree => newDegree);

    changeDegree(finalSeries, newDegree)
    
);
```


## Inversion Works now!
Main issue I guess was the displayed degree. It has to have the same displayed degree as the original LazySeries because when we multiply for testing, we want to get the 1 series. Obviously, the inverse is a LazySeries made up of LazySeries powers so of course we won't be able to calculate all of it, so extra degrees are going to give us extra stuff than just 1. Here are some examples. 

Additionally, `isUnit` works perfectly since below it did now allow a LazySeries with 0 constant to be inverted. Perfect!


In [17]:
loadPackage "LazySeriesPackage"
R = QQ[x]
M = lazySeries(R, i-> i+5)
L = lazySeries(R, i-> i, DisplayedDegree => 6, ComputedDegree => 6)


            2     3     4     5     6
o17 = x + 2x  + 3x  + 4x  + 5x  + 6x  + ... 

o17 : LazySeries


In [18]:
IM = inverse(M)


      1    6     1  2    4  3
o18 = - - --x + ---x  + ---x  + ... 
      5   25    125     625

o18 : LazySeries


In [19]:
IM * M


o19 = 1 + ... 

o19 : LazySeries


In [20]:
M* IM


o20 = 1 + ... 

o20 : LazySeries


In [21]:
IL = inverse(L)

stdio:38:6:(3): error: Cannot invert series because it is not a unit
